In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Raw = pd.read_csv('../input/titanic/train.csv')
print(Raw.head)
print(Raw.columns)

# Exploring Data

I'll try here to see which variables are important and which might be left behind. I'll start with sex and Pclass, since it just makes sence. Then I will see if I find others. 


In [ ]:
Raw['Sex'].value_counts().plot(kind='bar')
Raw['PassengerId']

In [ ]:
Raw.loc[Raw["Survived"] == 1, "Sex"].value_counts().plot(kind='bar')

Sex plays an important role. Must pasengers where male, but must survivors were female. Difference is not small. I conclude that this must be an important part of our model.

In [ ]:
Raw["Pclass"].value_counts().plot(kind='bar')

In [ ]:
Raw.loc[Raw["Survived"] == 1, "Pclass"].value_counts().plot(kind = "bar")

3rd class passengers outnumbered 1st and 2nd class passengers quite a bit in the original set. They are outnumbered by the former and almost matched by the later when we count only survivors. Pclass is, undoubtedly, an important variable. 

In [ ]:
Raw.groupby("Age").size().plot(kind='bar')
Raw.groupby("Age").describe()

In [ ]:
Raw.loc[Raw["Survived"] == 1].groupby("Age").size().plot(kind='bar')
Raw.loc[Raw["Survived"] == 1].groupby("Age").describe()

I don't see a clear tendency relating age with survival. I'll have to examine it in depth later. 

In [ ]:
Raw.groupby("SibSp").size().plot(kind = "bar")

In [ ]:
Raw.loc[Raw["Survived"] == 1].groupby("SibSp").size().plot(kind='bar')

It would seem that having at least one sibling/spouse improves your chances of survival, at least a little.

In [ ]:
Raw.groupby("Parch").size().plot(kind='bar')

In [ ]:
Raw.loc[Raw["Survived"] == 1].groupby("Parch").size().plot(kind='bar')

In [ ]:
Raw.groupby("Fare").size().plot(kind='bar')
Raw.groupby("Fare").describe()

In [ ]:
Raw.loc[Raw["Survived"]==1].groupby("Fare").size().plot(kind='bar')

In [ ]:
Raw.groupby("Embarked").size().plot(kind='bar')

In [ ]:
Raw.loc[Raw["Survived"]==1].groupby("Embarked").size().plot(kind='bar')

I will discrad embarked. 

In [ ]:
survival = Raw.Survived
survival_features = ['Sex','Pclass', 'SibSp', 'Parch']
X = Raw[survival_features]
X["Sex"].replace({'male': 1, 'female': 0}, inplace = True)
X.head

In [ ]:
from sklearn import svm
model = svm.SVC()
model.fit(X,survival)
testeo = pd.read_csv('../input/titanic/test.csv')
Y = testeo[survival_features]
Y["Sex"].replace({'male': 1, 'female': 0}, inplace = True)
ans = pd.DataFrame.from_dict({'PassengerId': np.array([i for i in range(892,1310)]), 'Survived': model.predict(Y)})
ans.to_csv('submission.csv',index=False)